In [6]:
import time
import warnings
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

warnings.filterwarnings("ignore") # uyarı mesajlarını ignore ediyoruz

# WebDriver ayarları
path = r"D:\PyCharm 2023.2.3\pythonProject1\notebook\chromedriver-win64\chromedriver.exe"
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")
service = Service(executable_path=path)
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)   # Maks 10 saniye beklemeye ayarlıyoruz 

 
# Hedef sayfalar ve her biri için sayfa sayısı
url_listesi = {
    "https://www.emlakjet.com/satilik-konut/bursa-nilufer": 50,
    "https://www.emlakjet.com/satilik-konut/bursa-osmangazi": 46,
    "https://www.emlakjet.com/satilik-konut/bursa-yildirim": 26,
    "https://www.emlakjet.com/satilik-konut/bursa-gemlik": 24,
    "https://www.emlakjet.com/satilik-konut/bursa-gursu": 6,
    "https://www.emlakjet.com/satilik-konut/bursa-inegol": 15,
    "https://www.emlakjet.com/satilik-konut/bursa-iznik": 3,            # Her ilçenin kaç sayfa verisi var ona göre hazırlanmış dict yapısı
    "https://www.emlakjet.com/satilik-konut/bursa-karacabey": 7,        # Sitenin 50.sayfadan sonra başa atma problemi var (ilk sayfa)
    "https://www.emlakjet.com/satilik-konut/bursa-kestel": 4,           # klon veri sayısını en aza indirmek ve niş veri sayısını arttırmak için
    "https://www.emlakjet.com/satilik-konut/bursa-mudanya": 21,
    "https://www.emlakjet.com/satilik-konut/bursa-mustafakemalpasa": 6,
    "https://www.emlakjet.com/satilik-konut/bursa-orhangazi": 5,
    "https://www.emlakjet.com/satilik-konut/bursa-yenisehir": 2
}

veriler = []

for url, max_sayfa in url_listesi.items():
    for sayfa in range(1, max_sayfa + 1):  # Sayfalar arasında gezinme, ilçeden ilçeye değişen sayfa sayısına göre 
        print(f"\n Sayfa {sayfa} işleniyor... {url}")
        driver.get(f"{url}?sayfa={sayfa}")
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.styles_wrapper__H_QG2")))  # Veri gelene kadar bekliyoruz
        ilanlar = driver.find_elements(By.CSS_SELECTOR, "a.styles_wrapper__H_QG2")


        for i, ilan in enumerate(ilanlar[:36], 1):  # Her sayfadaki verisi çekilecek ilan sayısı(her sayfada ortalama 35 ilan var)
            try:
                link = ilan.get_attribute("href")
                if not link:
                    continue

                driver.execute_script("window.open(arguments[0]);", link)
                driver.switch_to.window(driver.window_handles[1])

                wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="genel-bakis"]/div[1]/div[1]/div[1]/div/span')))

                # Temel bilgiler
                fiyat = driver.find_element(By.XPATH, '//*[@id="genel-bakis"]/div[1]/div[1]/div[1]/div/span').text
                net_m2 = driver.find_element(By.XPATH, '//*[@id="ilan-hakkinda"]/div/div[1]/ul/li[7]/span[2]').text
                brut_m2 = driver.find_element(By.XPATH, '//*[@id="ilan-hakkinda"]/div/div[1]/ul/li[8]/span[2]').text
                oda_sayisi = driver.find_element(By.XPATH, '//*[@id="ilan-hakkinda"]/div/div[1]/ul/li[9]/span[2]').text
                bina_yasi = driver.find_element(By.XPATH, '//*[@id="ilan-hakkinda"]/div/div[1]/ul/li[10]/span[2]').text
                kat = driver.find_element(By.XPATH, '//*[@id="ilan-hakkinda"]/div/div[1]/ul/li[11]/span[2]').text
                toplam_kat = driver.find_element(By.XPATH, '//*[@id="ilan-hakkinda"]/div/div[1]/ul/li[12]/span[2]').text
                isitma = driver.find_element(By.XPATH,'//*[@id="ilan-hakkinda"]/div/div[1]/ul/li[13]/span[2]').text

                # İl / İlçe / Mahalle
                konum = driver.find_element(By.XPATH, '//*[@id="genel-bakis"]/div[1]/div[1]/div[1]/span').text
                parcalar = konum.split(" - ")
                il = parcalar[0]
                ilce = parcalar[1]
                mahalle = parcalar[2].replace("Mahallesi", "").strip()

                # Site içinde mi?
                site_durumu = "Hayır"
                etiketler = driver.find_elements(By.CSS_SELECTOR, 'li.styles_featuresItem__HKN_I')
                for etiket in etiketler:
                    if "Site İçerisinde" in etiket.text or "Site" in etiket.text:
                        site_durumu = "Evet"
                        break

                # Veriyi ekle
                veriler.append({
                    "Fiyat": fiyat,
                    "Net m2": net_m2,
                    "Brüt m2": brut_m2,
                    "Oda Sayısı": oda_sayisi,
                    "Bina Yaşı": bina_yasi,
                    "Kat": kat,
                    "Toplam Kat": toplam_kat,
                    "Isıtma Tipi": isitma,
                    "İl": il,
                    "İlçe": ilce,
                    "Mahalle": mahalle,
                    "Site İçinde mi?": site_durumu
                })

                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                time.sleep(1)

            except Exception as e:
                print(f" Hata {i}. ilanda:", e)
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                continue

driver.quit()

# Excel çıktısı
df = pd.DataFrame(veriler)
df.to_excel("DENEMEVERİLER.xlsx", index=False, engine="openpyxl")
print("\n Veriler başarıyla kaydedildi: DENEMEVERİLER.xlsx")



 Sayfa 1 işleniyor... https://www.emlakjet.com/satilik-konut/bursa-nilufer

 Sayfa 2 işleniyor... https://www.emlakjet.com/satilik-konut/bursa-nilufer

 Sayfa 3 işleniyor... https://www.emlakjet.com/satilik-konut/bursa-nilufer

 Sayfa 4 işleniyor... https://www.emlakjet.com/satilik-konut/bursa-nilufer

 Sayfa 5 işleniyor... https://www.emlakjet.com/satilik-konut/bursa-nilufer

 Sayfa 6 işleniyor... https://www.emlakjet.com/satilik-konut/bursa-nilufer

 Sayfa 7 işleniyor... https://www.emlakjet.com/satilik-konut/bursa-nilufer

 Sayfa 8 işleniyor... https://www.emlakjet.com/satilik-konut/bursa-nilufer

 Sayfa 9 işleniyor... https://www.emlakjet.com/satilik-konut/bursa-nilufer

 Sayfa 10 işleniyor... https://www.emlakjet.com/satilik-konut/bursa-nilufer

 Sayfa 11 işleniyor... https://www.emlakjet.com/satilik-konut/bursa-nilufer

 Sayfa 12 işleniyor... https://www.emlakjet.com/satilik-konut/bursa-nilufer

 Sayfa 13 işleniyor... https://www.emlakjet.com/satilik-konut/bursa-nilufer

 Sayfa 